In [81]:
current = '/cm/archive/user6/bip/sbp/projects/bugsinpy-index.csv'
reproduce = '/cm/archive/user6/bip/sbp/projects/_bugsinpy-index.csv'

In [82]:
import pandas as pd

In [83]:
df_current = pd.read_csv(current)
df_reproduce = pd.read_csv(reproduce)

In [84]:
df_current.query("version == 'fixed' and result == 'pass'")

,repo,bugid,version,result
1,PySnooper,1,fixed,pass
8,httpie,1,fixed,pass
9,fastapi,1,fixed,pass
15,black,1,fixed,pass
16,fastapi,2,fixed,pass
...,...,...,...,...
875,pandas,166,fixed,pass
877,pandas,167,fixed,pass
879,pandas,168,fixed,pass
881,pandas,169,fixed,pass


In [85]:
df_reproduce.query("version == 'fixed' and result == 'pass'")

,repo,bugid,version,result
1,ansible,1,fixed,pass
3,ansible,2,fixed,pass
5,ansible,3,fixed,pass
7,ansible,4,fixed,pass
9,ansible,5,fixed,pass
...,...,...,...,...
993,youtube-dl,39,fixed,pass
995,youtube-dl,40,fixed,pass
997,youtube-dl,41,fixed,pass
999,youtube-dl,42,fixed,pass


In [86]:
df_reproduce.query("version == 'fixed'")

,repo,bugid,version,result
1,ansible,1,fixed,pass
3,ansible,2,fixed,pass
5,ansible,3,fixed,pass
7,ansible,4,fixed,pass
9,ansible,5,fixed,pass
...,...,...,...,...
993,youtube-dl,39,fixed,pass
995,youtube-dl,40,fixed,pass
997,youtube-dl,41,fixed,pass
999,youtube-dl,42,fixed,pass


In [87]:
gold_pred = '/cm/archive/user6/bip/p/bugsinpy_gold.jsonl'

In [88]:
import pandas as pd
preds = pd.read_json(gold_pred, lines=True)

In [89]:
allowed_ids = set(
    df_current
    .assign(instance_id=lambda x: x['repo'] + '-' + x['bugid'].astype(str))
    .query("version == 'fixed' and result == 'pass'")
    ['instance_id'].unique()
)

In [90]:
len(allowed_ids)

317

In [91]:
# preds['instance_id'].str.split('__').str[-1].str.split('-').str[:-1].str.join('-').value_counts()
preds['instance_id'].str.split('__').str[-1]

0          ansible-4
1         ansible-13
2          ansible-9
3          ansible-7
4         ansible-12
           ...      
496     youtube-dl-5
497    youtube-dl-16
498    youtube-dl-43
499    youtube-dl-24
500     youtube-dl-1
Name: instance_id, Length: 501, dtype: object

In [92]:
preds.query('instance_id.str.contains("spa")')

,instance_id,model_name_or_path,model_patch
391,explosion__spaCy-4,gold,diff --git a/spacy/cli/converters/conllu2json....
392,explosion__spaCy-9,gold,diff --git a/spacy/errors.py b/spacy/errors.py...
393,explosion__spaCy-7,gold,diff --git a/examples/information_extraction/e...
394,explosion__spaCy-6,gold,diff --git a/spacy/language.py b/spacy/languag...
395,explosion__spaCy-2,gold,diff --git a/spacy/util.py b/spacy/util.py\nin...
396,explosion__spaCy-3,gold,diff --git a/bin/wiki_entity_linking/wikipedia...
397,explosion__spaCy-10,gold,diff --git a/spacy/errors.py b/spacy/errors.py...
398,explosion__spaCy-8,gold,diff --git a/spacy/errors.py b/spacy/errors.py...
399,explosion__spaCy-5,gold,diff --git a/spacy/language.py b/spacy/languag...
400,explosion__spaCy-1,gold,diff --git a/spacy/errors.py b/spacy/errors.py...


In [103]:
# preds.query("instance_id.isin(@)")
(
    preds
    .assign(instance_id=lambda x: x['instance_id'].str.replace('spaCy', 'spacy'))
    .assign(id= lambda x: x['instance_id'].str.split('__').str[-1])
    .query('id.isin(@allowed_ids)')
    .drop(columns=['id'])
    .to_json('/cm/archive/user6/bip/p/bugsinpy_gold_fixed.jsonl', orient='records', lines=True)
)

In [102]:
(
    preds
    .assign(instance_id=lambda x: x['instance_id'].str.replace('spaCy', 'spacy'))
    .assign(id= lambda x: x['instance_id'].str.split('__').str[-1])
    .query('id.isin(@allowed_ids)')
    .drop(columns=['id'])
    .query('instance_id.str.contains("Snoo")')
    .assign(model_patch="")
    .to_json('/cm/archive/user6/bip/p/test.jsonl', orient='records', lines=True)
)

In [ ]:
with open('swesynth/lib/bugsinpy/reproducable_ids.txt', 'w') as f:
    f.write("\n".join(sorted(list(allowed_ids))))

In [ ]:
"""
All preds ->
1. filter 317
2. replace spacy
"""